In [1]:
import json
import numpy as np
from random import shuffle, choice
from itertools import product

IOIS = ['400', '600']
OCTAVES = ['3', '5', '7']
OFFSETS = ['-30', '-20', '-10', '0', '10', '20', '30']
CHROMATA = ['C', 'D', 'Ds', 'F', 'Fs', 'Gs', 'A', 'B']

NSESS = 300  # Number of sessions per group
NBLOCKS = 8  # Number of blocks per session

In [2]:
# Generate a block of 42 trials, with tempo changing every 7 trials
def randomize_block():
    """
    Each IOI x Octave x Offset should appear once per block, for a total of 42 trials. In order to
    change the base IOI every 7 trials, we define 6 "series" of 7 trials. One IOI will be assigned
    to series 0, 2, and 4 and one IOI to series 1, 3, and 5. We can therefore guarantee each of the
    42 trial types occurs once by generating two shuffled copies of all 21 octaves x offsets, and
    then assigning one set of 21 trials to the 3 "even" series and one set of 21 trials to the 3
    "odd" series.
    """
    global OCTAVES, OFFSETS
    
    n_series = 6
    trials_per_series = 7
    
    trials = [None for _ in range(n_series)]
    even_series = [[t[0], t[1]] for t in product(OCTAVES, OFFSETS)]
    odd_series = [[t[0], t[1]] for t in product(OCTAVES, OFFSETS)]
    shuffle(even_series)
    shuffle(odd_series)
    
    for i in range(n_series):
        if i % 2 == 0:
            # The indexing below is equivalent to: if i==0, 0:7; if i==2, 7:14; if i==4, 14:21
            trials[i] = [[x[0], 'X', x[1]] for x in even_series[trials_per_series*i//2:trials_per_series*(i//2+1)]]
        else:
            # The indexing below is equivalent to: if i==1, 0:7; if i==3, 7:14; if i==5, 14:21
            trials[i] = [[x[0], 'Y', x[1]] for x in odd_series[trials_per_series*(i-1)//2:trials_per_series*((i-1)//2+1)]]

    return trials

# Generate Trial Schedules

In [3]:
sessions = []
for i in range(NSESS):
    
    # Generate eight (valid) blocks for the session
    blocks = []
    while len(blocks) < NBLOCKS:
        new_block = randomize_block()
        blocks.append(new_block)
    session = np.concatenate(blocks)
    session = session.reshape((len(IOIS) * len(OCTAVES) * len(OFFSETS) * len(CHROMATA), 3))
    
    # Randomize which base rate occurs first, then fill it into the trial schedule (replacing 'X' and 'Y' placeholders)
    ioi_order = ['400', '600']
    shuffle(ioi_order)
    session[session=='X'] = ioi_order[0]
    session[session=='Y'] = ioi_order[1]
    
    # Randomly order the pitch classes within each octave
    for octave in OCTAVES:
        oct_mask = session[:, 0] == octave
        for ioi in IOIS:
            ioi_mask = session[:, 1] == ioi
            for offset in OFFSETS:
                offset_mask = session[:, 2] == offset
                
                # Randomly order the 8 pitch classes and slot them in to the 8 repetitions of each trial type
                shuffle(CHROMATA)
                session[oct_mask & ioi_mask & offset_mask, 0] = [c + str(octave) for c in CHROMATA]
    
    sessions.append(session)

### Sanity Checks

In [4]:
for i, session in enumerate(sessions):
    
    # Verify session length
    if len(session) != 336:
        raise ValueError('Session %i has improper length!' % i)
    
    # Verify condition counts
    for octave in OCTAVES:
        oct_mask = np.array([pitch[-1] == octave for pitch in session[:, 0]])
        for ioi in IOIS:
            ioi_mask = session[:, 1] == ioi
            for offset in OFFSETS:
                offset_mask = session[:, 2] == offset
                for chroma in CHROMATA:
                    chromask = np.array([pitch[:-1] == chroma for pitch in session[:, 0]])
                    
                    trials_of_type = np.sum(oct_mask & ioi_mask & offset_mask & chromask)
                    if trials_of_type != 1:
                        raise ValueError('Session %i has %i trials of octave %s, tempo %s, offset %s, and chroma %s!' % 
                                         (i, trials_of_type, octave, tempo, offset, chroma))

print('All sessions verified successfully!')

All sessions verified successfully!


### Save schedules

In [5]:
# Shuffle order of sessions and save
shuffle(sessions)
for i, session in enumerate(sessions):
    with open('../schedules/session%i.json' % (i+1), 'w') as f:
        json.dump(session.tolist(), f)